In [1]:
#Ignoring TF warning messages
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
(train_data, validation_data, test_data), info = tfds.load('cats_vs_dogs',
                                                           split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                                           with_info=True,
                                                           as_supervised=True)

In [4]:
IMG_SIZE = 150

def format_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize the pixel values
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize to the desired size
    return image, label
batch_size = 32
train_data = train_data.map(format_image).shuffle(1000).batch(batch_size)
validation_data = validation_data.map(format_image).batch(batch_size)
test_data = test_data.map(format_image).batch(batch_size)

In [5]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [6]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']) 

In [7]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

In [8]:
with tf.device('GPU:0'):
    history = model.fit(train_data,
                        epochs=3,
                        validation_data=validation_data)

Epoch 1/3
  5/582 [..............................] - ETA: 15s - loss: 0.5116 - accuracy: 0.7312   

I0000 00:00:1705648453.909564   32082 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


582/582 [==============================] - 25s 32ms/step - loss: 0.1173 - accuracy: 0.9510 - val_loss: 0.0975 - val_accuracy: 0.9630
Epoch 2/3
582/582 [==============================] - 17s 28ms/step - loss: 0.0881 - accuracy: 0.9645 - val_loss: 0.0921 - val_accuracy: 0.9695
Epoch 3/3
582/582 [==============================] - 17s 29ms/step - loss: 0.0795 - accuracy: 0.9676 - val_loss: 0.0955 - val_accuracy: 0.9665


In [28]:
test_loss, test_accuracy = model.evaluate(test_data)
print('test_acc:', test_accuracy) #

73/73 [==============================] - 2s 28ms/step - loss: 0.0847 - accuracy: 0.9686
test_acc: 0.9686156511306763
